In [14]:
import pandas as pd

In [15]:
df = pd.read_csv("naive bayes.csv")

In [16]:
df

,Dia,Tempo,Temperatura,Humidade,Vento,Jogar
0,1,Sol,Quente,Alta,Fraco,Não
1,2,Sol,Quente,Alta,Forte,Não
2,3,Coberto,Quente,Alta,Fraco,Sim
3,4,Chuva,Normal,Alta,Fraco,Sim
4,5,Chuva,Frio,Normal,Fraco,Não
5,6,Chuva,Frio,Normal,Forte,Não
6,7,Coberto,Frio,Normal,Forte,Sim
7,8,Sol,Normal,Alta,Fraco,Não
8,9,Sol,Frio,Normal,Fraco,Sim
9,10,Chuva,Normal,Normal,Fraco,Sim


In [17]:
df = df.set_index('Dia')

In [18]:
x = df['Jogar'].value_counts()

In [19]:
f = lambda x: f"{x}/{len(df)}"
P_ys = {}
y_counts = df['Jogar'].value_counts()
for value,num in y_counts.items():
    print(value,' = ',f(num))
    P_ys[value] = num/len(df)


Não  =  5/10
Sim  =  5/10


In [26]:

target = 'Jogar'
for key in df.keys():
    if key == target:
        continue
    print(pd.crosstab(df[key],df.Jogar,margins=True))
    print('-'*20)

Jogar    Não  Sim  All
Tempo                 
Chuva      2    2    4
Coberto    0    2    2
Sol        3    1    4
All        5    5   10
--------------------
Jogar        Não  Sim  All
Temperatura               
Frio           2    2    4
Normal         1    2    3
Quente         2    1    3
All            5    5   10
--------------------
Jogar     Não  Sim  All
Humidade               
Alta        3    2    5
Normal      2    3    5
All         5    5   10
--------------------
Jogar  Não  Sim  All
Vento               
Forte    2    1    3
Fraco    3    4    7
All      5    5   10
--------------------


In [20]:

target = 'Jogar'

P_x_y = pd.Series(dtype='float64')


for key in df.keys():
    if key == target:
        continue
    
    df_aux = df[[key,'Jogar']].groupby([key,'Jogar']).value_counts()
    P_xs = []
    for (key,jogar),value in df_aux.items():
        P_xs.append((value+1)/(y_counts[jogar]+2)) # esse +1 e +2 é o laplace smoothing conhecimento proibidissimo


    df_aux = pd.Series(P_xs,index=df_aux.index)
    P_x_y = pd.concat([P_x_y,df_aux])
    print(df_aux)
    
    print("-"*50)

Tempo    Jogar
Chuva    Não      0.428571
         Sim      0.428571
Coberto  Sim      0.428571
Sol      Não      0.571429
         Sim      0.285714
dtype: float64
--------------------------------------------------
Temperatura  Jogar
Frio         Não      0.428571
             Sim      0.428571
Normal       Não      0.285714
             Sim      0.428571
Quente       Não      0.428571
             Sim      0.285714
dtype: float64
--------------------------------------------------
Humidade  Jogar
Alta      Não      0.571429
          Sim      0.428571
Normal    Não      0.428571
          Sim      0.571429
dtype: float64
--------------------------------------------------
Vento  Jogar
Forte  Não      0.428571
       Sim      0.285714
Fraco  Não      0.571429
       Sim      0.714286
dtype: float64
--------------------------------------------------


In [60]:
# Calculando se for ter Jogo:
Tempo = P_x_y[('Chuva','Sim')]
Temperatura = P_x_y[('Quente','Sim')]
try:
    Humidade = P_x_y[('Baixa','Sim')]
except:
    Humidade = 1/(y_counts["Sim"]+2)# esse +1 e +2 é o laplace smoothing conhecimento proibido 
Vento = P_x_y[('Forte','Sim')]

Tempo*Temperatura*Humidade*Vento*P_ys['Sim']


0.002498958767180341

In [61]:
# Calculando se for ter Jogo:
Tempo = P_x_y[('Chuva','Não')]
Temperatura = P_x_y[('Quente','Não')]
try:
    Humidade = P_x_y[('Baixa','Não')]
except:
    Humidade = 1/(y_counts['Não']+2) # esse +1 e +2 é o laplace smoothing conhecimento proibido 
Vento = P_x_y[('Forte','Não')]

Tempo*Temperatura*Humidade*Vento*P_ys['Não']

0.005622657226155767